# Problem Description:
Boston’s transportation system, the [MBTA](https://mbta.com/), has a [website with APIs](https://api-v3.mbta.com/docs/swagger/index.html).

## Setup

I decided to add the API key because a ratelimit of 20 was turning out to be pretty low, and causing KeyErrors in some of my loops

In [29]:
import requests

api_key = "6f1058bc4865463e8aba8608821ab32c"
default_params = {"api_key": api_key}
endpoint_routes = "https://api-v3.mbta.com/routes"
endpoint_stops = "https://api-v3.mbta.com/stops"

Checking my ratelimit

In [2]:
r = requests.get(endpoint_stops, params = default_params )
print(r.headers)

{'Date': 'Wed, 20 Oct 2021 04:14:23 GMT', 'Content-Type': 'application/vnd.api+json; charset=utf-8', 'Content-Length': '545287', 'Connection': 'keep-alive', 'cache-control': 'max-age=0, private, must-revalidate', 'content-encoding': 'gzip', 'last-modified': 'Tue, 19 Oct 2021 22:17:05 GMT', 'server': 'Cowboy', 'strict-transport-security': 'max-age=31536000', 'vary': 'accept-encoding', 'x-ratelimit-limit': '1000', 'x-ratelimit-remaining': '999', 'x-ratelimit-reset': '1634703300', 'x-request-id': 'Fq-h9MWnIzZT33ABOtKC'}


## 1) Write a program that retrieves data representing all “subway” routes:
“Light Rail” (type 0) and “Heavy Rail” (type 1). The program should list their “long names”
on the console.

-----
_This is a lot of output and can also be seen in the [Swagger UI](https://api-v3.mbta.com/docs/swagger/index.html#/) - but I found it useful to see how it looked here (as it was not as pretty)_

In [ ]:
# requests.get(endpoint_routes, params = default_params).json()

I chose to filter the routes directly in the request
"https://api-v3.mbta.com/routes?filter[type]=0,1" because then I'm only retrieving the data I want* for this exercise and I don't have to write additional logic to do this.

_*which is basically the whole point of an API to begin with_

In [ ]:
# filter_params = {"fields[route]": "short_name,long_name,id", "filter[type]": "1,0", "api_key": api_key}
# requests.get(endpoint_routes, filter_params).json()

### get_route_names()
returns long_names of "Subway" routes

In [30]:
def get_route_names():
    filter_params = {"fields[route]": "short_name,long_name,id", "filter[type]": "1,0", "api_key": api_key}
    response_mbta = requests.get(endpoint_routes, filter_params).json()
    data = response_mbta["data"]
    for route in data:
        # print(route)
        # print(route['attributes']['long_name'])
        name = route['attributes']['long_name']
        print(name)
        
        
get_route_names()

Red Line
Mattapan Trolley
Orange Line
Green Line B
Green Line C
Green Line D
Green Line E
Blue Line


## 2) Extend your program so it displays the following additional information.
    a) The name of the subway route with the most stops as well as a count its stops.
    b) The name of the subway route with the fewest stops as well as a count its stops.
    c) A list of the stops that connect two or more subway routes along with the relevant route names for each of those stops


Get the list of long_names & route_ids from the routes endpoint and create a tuple, which we can use later to loop over and create our Route class objects.

In [31]:
long_names = []
route_ids = []

def get_route_names():
    response_routes = requests.get(endpoint_routes, filter_params).json()
    data_routes = response_routes["data"]
    for route in data_routes:
        long_name = route['attributes']['long_name']
        route_id = route['id']
        long_names.append(long_name)
        route_ids.append(route_id)
    name_tuple = zip(long_names, route_ids)
    return name_tuple
        
route_names_ids = get_route_names()
print("long names: " + str(long_names))
print("route ids" + str(route_ids))

long names: ['Red Line', 'Mattapan Trolley', 'Orange Line', 'Green Line B', 'Green Line C', 'Green Line D', 'Green Line E', 'Blue Line']
route ids['Red', 'Mattapan', 'Orange', 'Green-B', 'Green-C', 'Green-D', 'Green-E', 'Blue']


---
Define a function to get the list of stops for the specified route (by route id)

In [44]:
## Remember that route == route_id (note to self)

def get_stops(route_id):
    endpoint_stops = "https://api-v3.mbta.com/stops"
    stop_q_params = {"fields[stop]": "name", "include": "route", "filter[route]": route_id, "api_key": api_key}
    response_stops = requests.get(endpoint_stops, stop_q_params).json()
    #print(response_stops)
    stop_data = response_stops["data"]
    route_stops = []
    for stop in stop_data:
        stop_name = stop["attributes"]["name"]
        route_stops.append(stop_name)
        # print(stop_name)
    return route_stops

#get_stops("Red")
print("-- TESTING get_stops() --")
print(get_stops("Red"))

-- TESTING get_stops() --
['Alewife', 'Davis', 'Porter', 'Harvard', 'Central', 'Kendall/MIT', 'Charles/MGH', 'Park Street', 'Downtown Crossing', 'South Station', 'Broadway', 'Andrew', 'JFK/UMass', 'Savin Hill', 'Fields Corner', 'Shawmut', 'Ashmont', 'North Quincy', 'Wollaston', 'Quincy Center', 'Quincy Adams', 'Braintree']


---
Now, make a dictionary with routes as keys and their list of stops as the values, by looping over get_stops()

In [45]:
route_dict = {}
def make_route_stop_dict():    
    for route_id in route_ids:
        #print("{route_id} stops are {route_stops}".format(route_id = route_id, route_stops = get_stops(route_id)))
        route_dict[route_id] = get_stops(route_id)
    return route_dict

make_route_stop_dict()
print(route_dict)


{'Red': ['Alewife', 'Davis', 'Porter', 'Harvard', 'Central', 'Kendall/MIT', 'Charles/MGH', 'Park Street', 'Downtown Crossing', 'South Station', 'Broadway', 'Andrew', 'JFK/UMass', 'Savin Hill', 'Fields Corner', 'Shawmut', 'Ashmont', 'North Quincy', 'Wollaston', 'Quincy Center', 'Quincy Adams', 'Braintree'], 'Mattapan': ['Ashmont', 'Cedar Grove', 'Butler', 'Milton', 'Central Avenue', 'Valley Road', 'Capen Street', 'Mattapan'], 'Orange': ['Forest Hills', 'Green Street', 'Stony Brook', 'Jackson Square', 'Roxbury Crossing', 'Ruggles', 'Massachusetts Avenue', 'Back Bay', 'Tufts Medical Center', 'Chinatown', 'Downtown Crossing', 'State', 'Haymarket', 'North Station', 'Community College', 'Sullivan Square', 'Assembly', 'Wellington', 'Malden Center', 'Oak Grove'], 'Green-B': ['Boston College', 'South Street', 'Chestnut Hill Avenue', 'Chiswick Road', 'Sutherland Road', 'Washington Street', 'Warren Street', 'Allston Street', 'Griggs Street', 'Harvard Avenue', 'Packards Corner', 'Babcock Street', 

---
### Now let's put it together

At this point, I'm just going to be recreating parts of the API if I make more dictionaries, so it's time to create a Route class

In [46]:
class Route():
    
    def __init__(self, long_name, route_id, stops):
        self.long_name = long_name
        self.route_id = route_id
        self.stops = stops
        self.num_stops = len(stops)
        
    def __repr__(self):
        return str(self)


And now we loop over the list of tuples created with get_route_names, and grab the list of stops for each route (values in the route_dict) to create Route class objects for each route 

In [35]:
#list of class objects
routeList = []
for long_name, route_id in route_names_ids:
    routeList.append(Route(long_name, route_id, route_dict[route_id]))

# TESTING
# for route in routeList:
#     print('long_name: {}, route_id: {}, stops: {}'.format(route.long_name, route.route_id, route.stops))
        

(and print out the instance variables for num_stops & route_id to test before moving on)

_Programmatically, this could have been done by checking for empty values_

In [36]:
for route in routeList:
    print(route.num_stops)
    print(route.route_id)

22
Red
8
Mattapan
20
Orange
23
Green-B
22
Green-C
19
Green-D
18
Green-E
12
Blue


## 2) answers a + b
    a) The name of the subway route with the most stops as well as a count its stops.
    b) The name of the subway route with the fewest stops as well as a count its stops.

In [50]:
# doesn't account for a tie

def count_stops():
    most_stops = 0
    fewest_stops = 100
    most_stops_name = ""
    fewest_stops_name = ""
    for route in routeList:
        if route.num_stops >= most_stops:
            most_stops = route.num_stops
            most_stops_name = route.long_name
        elif route.num_stops <= fewest_stops:
            fewest_stops = route.num_stops
            fewest_stops_name = route.long_name
    print("{route} has the most stops with {num} stops".format(route = most_stops_name, num = most_stops))
    print("{route} has the fewest stops with {num} stops".format(route = fewest_stops_name, num = fewest_stops))

count_stops()

Green Line B has the most stops with 23 stops
Mattapan Trolley has the fewest stops with 8 stops


 ## 2 c) 
A list of the stops that connect two or more subway routes along with the relevant route names for each of those stops


Started by getting a list of all the stops that connect two or more subway routes

In [61]:
def get_connecting_stops():
    #List of stops
    all_stop_lists = list(route_dict.values())
    all_stop_route_list = list(route_dict.keys())

    ## TESTING ## 
    # print(all_stop_lists)
    # red_list = list(route_dict.get('Red'))
    # green_blist = list(route_dict.get('Green-B'))
    # print("RED LIST")
    # print(red_list)
    # print("GREEN B LIST")
    # print(green_blist)
    #############

    connecting_stops = []

    for i in range(len(all_stop_lists)):
        for j in range(i + 1, len(all_stop_lists)):
            stop_match = set(all_stop_lists[i]) & set(all_stop_lists[j])
            if len(stop_match) > 0:
                # print(all_stop_route_list[i])
                # print(list(stop_match))
                connecting_stops += list(stop_match)

    # print(connecting_stops)

    # clean up duplicates
    connecting_stops_unique = []
    for stop in connecting_stops:
        if stop not in connecting_stops_unique:
            connecting_stops_unique.append(stop)
    # print(connecting_stops_unique)
    return connecting_stops_unique

print(get_connecting_stops())
connection_stops = get_connecting_stops()

['Ashmont', 'Downtown Crossing', 'Park Street', 'Haymarket', 'North Station', 'State', 'Saint Paul Street', 'Boylston', 'Hynes Convention Center', 'Arlington', 'Kenmore', 'Copley', 'Government Center']


In [64]:
# print('----by route----')
# for route in routeList:
#     rte_match = set(route.stops) & set(connection_stops)
#     if len(rte_match) > 0:
#         print(route.long_name)
#         print(list(rte_match))
        

# flip it and reverse it...
print("---By stop ---")

def stops_and_connected_routes():
    for stop in connection_stops:
        cnct_stops_by_route = []
        for route in routeList:
            if stop in route.stops:
                cnct_stops_by_route.append(route.long_name)
        print(stop)
        print(cnct_stops_by_route)

stops_and_connected_routes()

---By stop ---
Ashmont
['Red Line', 'Mattapan Trolley']
Downtown Crossing
['Red Line', 'Orange Line']
Park Street
['Red Line', 'Green Line B', 'Green Line C', 'Green Line D', 'Green Line E']
Haymarket
['Orange Line', 'Green Line C', 'Green Line E']
North Station
['Orange Line', 'Green Line C', 'Green Line E']
State
['Orange Line', 'Blue Line']
Saint Paul Street
['Green Line B', 'Green Line C']
Boylston
['Green Line B', 'Green Line C', 'Green Line D', 'Green Line E']
Hynes Convention Center
['Green Line B', 'Green Line C', 'Green Line D']
Arlington
['Green Line B', 'Green Line C', 'Green Line D', 'Green Line E']
Kenmore
['Green Line B', 'Green Line C', 'Green Line D']
Copley
['Green Line B', 'Green Line C', 'Green Line D', 'Green Line E']
Government Center
['Green Line C', 'Green Line D', 'Green Line E', 'Blue Line']


## 3) Extend your program again such that the user can provide any two stops on the subway routes you listed for question 1.

List the rail routes you would travel to get from one stop to the other. Pick a simple solution that answers the question.

---
This is where I ran out of time - I'm able to get routes that only use one or two T routes. I'm missing the connecting routes for those that require a 3rd or 4th route to get from point A to point B. 

In [65]:
#misses those routes which need a connecting stop in the middle - IE Red to Blue, needs Green or Orange
def which_way(start, stop):
    the_way = []
    route_start = ""
    route_stop = ""
    for route in routeList:
        #this is good - but only works for this case
        if start in route.stops and stop in route.stops:
            the_way.append(route.long_name)
            return(the_way)
        elif start in route.stops:
            route_start = route.long_name
        elif stop in route.stops:
            route_stop= route.long_name
    the_way.append(route_start)
    the_way.append(route_stop)
    return(the_way)




#Test
print(which_way("Park Street", "Central"))
print(which_way("Ashmont", "Kenmore" ))
#misses needed connection
print(which_way("Alewife", "State"))
            

['Red Line']
['Mattapan Trolley', 'Green Line D']
['Red Line', 'Blue Line']
